In [ ]:
%pip install -q -U google-generativeai nltk

In [ ]:
import google.generativeai as genai
import json
import re
import nltk
from nltk.corpus import stopwords
from google.colab import userdata

ModuleNotFoundError: No module named 'google.generativeai'

In [ ]:
# Download de recursos para NLP
nltk.download('stopwords')
stop_words = set(stopwords.words('portuguese'))


try:
    API_KEY = userdata.get('GEMINI_API_KEY')
    genai.configure(api_key=API_KEY)
    model = genai.GenerativeModel('gemini-3-flash-preview')
    print("Setup concluído com sucesso!")
except:
    print("Configure sua GEMINI_API_KEY nos Secrets do Colab.")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Setup concluído com sucesso!


In [ ]:
def pre_processar_texto(texto):
    if not texto: return ""

    # 1. Padronização para minúsculo
    texto = texto.lower()

    # 2. Remoção de URLs e caracteres especiais (mantendo acentos)
    texto = re.sub(r'http\S+|www\S+|https\S+', '', texto)
    texto = re.sub(r'[^a-zA-Záàâãéèêíïóôõöúçñ1-9\s]', '', texto)

    # 3. Remoção de Stop Words
    palavras = texto.split()
    palavras_filtradas = [w for w in palavras if w not in stop_words]

    return " ".join(palavras_filtradas)

# Teste rápido
print(f"Exemplo limpo: {pre_processar_texto('Olá! Segue o boleto em anexo.')}")

Exemplo limpo: olá segue boleto anexo


In [ ]:
def classificar_email(texto_limpo):
    prompt = f"""
    Classifique o email financeiro abaixo em uma destas categorias:
    - 'Produtivo': Requisições, dúvidas técnicas, status de boletos ou envio de arquivos.
    - 'Improdutivo': Saudações, agradecimentos, mensagens festivas ou spans.

    Texto: {texto_limpo}

    Retorne APENAS um JSON no formato: {{"categoria": "Produtivo ou Improdutivo"}}
    """

    try:
        response = model.generate_content(prompt)
        # Limpa formatação de markdown se houver
        json_text = response.text.replace('```json', '').replace('```', '').strip()
        return json.loads(json_text)
    except Exception as e:
        return {"erro": str(e)}

In [ ]:
def sugerir_resposta(texto_limpo, categoria):
    if categoria == "Improdutivo":
        return "Agradecemos o contato. Desejamos um excelente dia!"
    prompt = f"""
    Você é um assistente de marketing financeiro da empresa AutoU.
    Gere uma resposta curta e profissional para um cliente financeiro.
    O email dele tratava de: {texto_limpo}
    A resposta deve ser empática e informar que o time técnico já está analisando.
    Apenas uma opção.
    """

    try:
        response = model.generate_content(prompt)
        return response.text.strip()
    except:
        return "Recebemos sua mensagem e daremos retorno em breve."

In [ ]:
# Simulando a entrada de um email
email_entrada = "Bom dia time! Gostaria de uma atualização sobre o meu processo de reembolso."

# 1. Limpeza
texto_limpo = pre_processar_texto(email_entrada)

# 2. Classificação
classificacao_dict = classificar_email(texto_limpo)
categoria = classificacao_dict.get("categoria", "Não Identificado")

# 3. Resposta
resposta = sugerir_resposta(texto_limpo, categoria)

# Resultado Final
print(f"--- RESULTADO DA ANÁLISE ---")
print(f"Categoria: {categoria}")
print(f"Resposta Sugerida: {resposta}")

--- RESULTADO DA ANÁLISE ---
Categoria: Não Identificado
Resposta Sugerida: Olá, [Nome do Cliente].

Compreendemos a importância do seu pedido e agradecemos a paciência. Informamos que o seu processo de reembolso já está em análise prioritária pelo nosso time técnico para que tudo seja resolvido o quanto antes.

Em breve, entraremos em contato com uma atualização.

Atenciosamente,
**Equipe AutoU**


In [ ]:
%pip install PyPDF2 -q
import PyPDF2
import io

def extrair_texto_arquivo(file_bytes, filename):
    """Extrai texto de arquivos PDF ou TXT enviados pelo usuário."""
    if filename.endswith('.pdf'):
        try:
            pdf_reader = PyPDF2.PdfReader(io.BytesIO(file_bytes))
            texto = ""
            for page in pdf_reader.pages:
                texto += page.extract_text() or ""
            return texto
        except Exception as e:
            return f"Erro ao ler PDF: {e}"
    elif filename.endswith('.txt'):
        return file_bytes.decode("utf-8")
    return ""

In [ ]:
%pip install fastapi uvicorn python-multipart nest-asyncio pyngrok -q
from fastapi import FastAPI, UploadFile, File, Form
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio
import time

app = FastAPI()

# Configuração de CORS para permitir que seu site acesse o backend
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.post("/analisar")
async def api_analisar_email(
    texto: str = Form(None),
    arquivo: UploadFile = File(None)
):
    # 1. Captura o conteúdo bruto (seja texto direto ou de arquivo)
    email_bruto = ""
    if arquivo:
        file_bytes = await arquivo.read()
        email_bruto = extrair_texto_arquivo(file_bytes, arquivo.filename)
    else:
        email_bruto = texto

    if not email_bruto:
        return {"erro": "Nenhum conteúdo recebido."}

    # 2. Orquestração das funções que você já criou
    texto_limpo = pre_processar_texto(email_bruto)
    resultado_classe = classificar_email(texto_limpo)
    categoria = resultado_classe.get("categoria", "Não Identificado")
    resposta = sugerir_resposta(texto_limpo, categoria)

    return {
        "categoria": categoria,
        "resposta_sugerida": resposta
    }

In [106]:
# Célula de Teste de Qualidade
emails_para_teste = [
    "Olá, segue em anexo o comprovante do boleto pago ontem. Favor confirmar o recebimento.", # Esperado: Produtivo
    "Bom dia! Gostaria de saber por que minha taxa de manutenção aumentou este mês.", # Esperado: Produtivo
    "Feliz Natal para toda a equipe da AutoU! Vocês são nota 10.", # Esperado: Improdutivo
    "Obrigado pelo retorno, tenha um bom dia!", # Esperado: Improdutivo
    "PRECISO DE AJUDA! Meu sistema travou e não consigo emitir a nota fiscal agora." # Esperado: Produtivo
]

print(f"{'EMAIL ORIGINAL':<50} | {'CATEGORIA':<15} | {'RESPOSTA OK?'}")
print("-" * 85)

for email in emails_para_teste:
    time.sleep(4)
    limpo = pre_processar_texto(email)
    classe = classificar_email(limpo)
    cat = classe.get("categoria", "Erro")
    resp = sugerir_resposta(limpo, cat)

    # Verifica se a resposta contém palavras profissionais esperadas
    valido = "✅" if len(resp) > 10 else "❌"
    print(f"{email[:48]:<50}... | {cat:<15} | {valido}")

EMAIL ORIGINAL                                     | CATEGORIA       | RESPOSTA OK?
-------------------------------------------------------------------------------------
Olá, segue em anexo o comprovante do boleto pago  ... | Produtivo       | ✅
Bom dia! Gostaria de saber por que minha taxa de  ... | Erro            | ✅
Feliz Natal para toda a equipe da AutoU! Vocês s  ... | Improdutivo     | ✅


ERROR:tornado.access:503 POST /v1beta/models/gemini-3-flash-preview:generateContent?%24alt=json%3Benum-encoding%3Dint (::1) 2718.91ms


Obrigado pelo retorno, tenha um bom dia!          ... | Improdutivo     | ✅


PRECISO DE AJUDA! Meu sistema travou e não consi  ... | Erro            | ✅
